In [3]:
import warnings
warnings.filterwarnings('ignore')

## import - 문제 발생 여지

In [4]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
from datetime import datetime
import urllib
import requests as rq
import urllib.request as ulq

## 과제0727_2
- 네이버 뉴스 검색 사이트에서 검색 키워드와 총 필요한 뉴스기사 수를 입력하면 요청한 만큼 데이터 프레임 형태(컬럼은 title, url)로 출력해주는 크롤링 프로그램을 수행하세요.
-https://search.naver.com/search.naver?where=news&sm=tab_jum&query=

- 기사는 각 페이지에 10개
- 네이버는 뉴스 검색결과를 4000건 까지 제공

- query= (검색어, 한글도 그냥 가능)
- start=(1, 11, 21,... 로 page 결정 가능)
- (cluster_rank=이건 바꿔도 아무일도 일어나지 않는다.)

- 검색 결과가 없을때: 
```
<div class="not_found02">
...
</div>
```

- 기사 제목과 링크 :
```
<a class="news_tit" 
href="https://www.joongang.co.kr/article/25089380" 
onclick="return goOtherCR(this, 'a=nws*a.tit&amp;r=1&amp;i=880000F2_000000000000000003211962&amp;g=025.0003211962&amp;u='+urlencode(this.href));" target="_blank" 
title="[소년중앙] 친구를 위해서라면 우주 모험도 할 수 있어 ‘극장판 도라에몽: 진구의 우주소전쟁’">
[소년중앙] 친구를 위해서라면 우주 모험도 할 수 있어 ‘극장판
<mark>
도라에몽
</mark>
: 진...
</a>
```

In [13]:
def getLinksFrom(url, fw):
        df = pd.DataFrame(columns=['title', 'url'])
        idx = 0
        
        response = requests.get(url)
        fw.write(f'********** status_code : {response.status_code}\n')
            
        if response.status_code != 200:
            return None
        
        html = response.text
        bs = BeautifulSoup(html, 'html.parser')
        
        # 검색결과가 하나도 없는 경우
        notFound = bs.find_all('div', {'class' : "not_found02"})
        if notFound:
            fw.write(f'********** keyword not found\n')
            return None, None
        
        tagA = bs.find_all('a', {'class': "news_tit"})
        fw.write(f'********** len(tagA) : {len(tagA)}\n')
        if not tagA:
            return None, None
  
        for a in tagA:
            title = a.get("title")
            href = a.get("href")
            fw.write(f'********** title : {title}\n')
#             try:
#                 fw.write(f'********** title : {title}\n')
#             except UnicodeEncodeError:
#                 fw.write(f'********** title : UnicodeEncodeError\n')
            fw.write(f'********** href : {href}\n')
            df.loc[idx] = [title, href]
            idx += 1
        return idx,  df

In [15]:
keyword = input("검색할 단어>")

while True:
    nNews = input("필요한 링크 갯수>")
    
    try:
        nNews = int(nNews)
    except ValueError:
        print("숫자로 입력")
        continue
        
    if (nNews >= 1) and (nNews <= 4000):
        break
    else:
        print("1~4000 입력")

검색할 단어>강기윤
필요한 링크 갯수>500


In [16]:
# 퍼센트 인코딩
# 안해도 되네.?

In [17]:
dt = datetime.now()
dt = dt.strftime('_%m%d_%H%M')
df_sum = pd.DataFrame(columns=['title', 'url'])
p=0
nLeft = nNews - df_sum.shape[0]

with open(f'./logs/log_for_news_url{dt}.txt', 'w', encoding='utf-8') as fw:
    while nLeft > 0:
        fw.write(f'***** for page : {p + 1}\n')
        start = p*10+1
        url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={start}'
        fw.write(f'***** url : {url}\n')
        
        # getLinksFrom(url, fw) : 페이지 url을 넣으면 기사 DataFrame을 반환 [title,url], 행이 10개 실패시 None을 반환
        rows, df = getLinksFrom(url, fw)
        fw.write(f'***** row : {rows}\n')
        
        # 기사링크 못 받아오면
        if not rows: 
            fw.write(f'***** fail to get titles and links from : {url}\n')
            fw.write(f'***** get only {df_sum.shape[0]} rows.\n')
            break
        elif rows == 1: # 기사 링크를 1개만 받아오면
            df_sum = pd.concat([df_sum, df])
            fw.write(f'***** get only {df_sum.shape[0]} rows.\n')
            break
        else: # 기사 링크를 2개 이상 받아오면
            df_sum = pd.concat([df_sum, df])
            p += 1
            nLeft = nNews - df_sum.shape[0]
            print(nLeft, end= " ")
print('\n')
df_sum.reset_index(drop=True, inplace=True)          
if nLeft < 0:
    df_sum.drop(list(range(nNews, nNews-nLeft)), inplace=True)
df_sum.info()
df_sum.drop_duplicates(inplace=True)
df_sum.info()

490 480 470 460 450 440 430 420 410 400 390 380 370 360 350 340 330 320 310 300 290 280 270 260 250 240 230 220 210 200 190 180 170 160 150 140 130 120 110 100 90 80 70 60 50 40 30 20 10 0 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   500 non-null    object
 1   url     500 non-null    object
dtypes: object(2)
memory usage: 7.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 362 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   362 non-null    object
 1   url     362 non-null    object
dtypes: object(2)
memory usage: 8.5+ KB


In [20]:
df_sum.to_excel('gyGang_test.xlsx')

In [28]:
print(df_sum.loc[:100])

                                                title  \
0                      강기윤 "확진자 접촉 무증상자도 신속항원 검사 무료화"   
1                           보건복지위원장 정춘숙…간사 강기윤‧강훈식 의원   
2   "팬데믹 시대 과학 방역의 핵심, 관료 아닌 전문가에 맡길 것" [안철수 민·당·정...   
3                   강기윤 '코로나 재유행대비 의료체계 점검 긴급 간담회' 연다   
4                  강기윤"코로나 재유행대비 의료체계 점검 위한 긴급 간담회"개최   
..                                                ...   
95                            강기윤 의원, “마산 바닷길 다시 열겠다”   
96                정호영 후보자 아들 MRI 관련 언쟁 벌이는 김성주-강기윤 간사   
97                                    논의하는 김성주-강기윤 의원   
98                        강기윤, '과다 보상금' 1억여 원 창원시에 환급   
99                       강기윤 의원, ‘허위 보상’ 1억 원 창원시에 반납   

                                                  url  
0   http://www.edaily.co.kr/news/newspath.asp?news...  
1   https://www.dailymedi.com/news/news_view.php?w...  
2   http://www.sisaon.co.kr/news/articleView.html?...  
3   http://daily.hankooki.com/news/articleView.htm...  
4   https://www.gukjenews.com/news/